# Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading dataset**

In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
import glob
from keras.preprocessing.image import ImageDataGenerator




img_directory="/content/drive/MyDrive/Colab Notebooks/fetal training set"
mask_directory="/content/drive/MyDrive/Colab Notebooks/fetal head annotation"
size=256
num_img=999

In [ ]:
image_names=glob.glob("/content/drive/MyDrive/Colab Notebooks/fetal training set/*_HC.png")
print(image_names)
image_names.sort()
print(image_names)
print(len(image_names))
image_names_subset=image_names[0:num_img]
images=[cv2.imread(img,3) for img in image_names_subset]
images=[cv2.resize(img,(256,256)) for img in images]
image_dataset=np.array(images)
#image_dataset = np.expand_dims(image_dataset, axis = 3)
print(image_dataset.shape)

['/content/drive/MyDrive/Colab Notebooks/fetal training set/000_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/001_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/002_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/003_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/007_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/006_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/008_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/005_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/004_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/009_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/012_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/010_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/011_HC.png', '/content/drive/MyDrive/Colab Notebooks/fetal training set/014_

In [ ]:
mask_names=glob.glob("/content/drive/MyDrive/Colab Notebooks/fetal head annotation/*.png")
mask_names.sort()
mask_names_subset=mask_names[0:num_img]
masks=[cv2.imread(mask,0) for mask in mask_names_subset]
masks=[cv2.resize(mask,(256,256)) for mask in masks]
mask_dataset=np.array(masks)
mask_dataset=np.expand_dims(mask_dataset, axis=3)
print(mask_dataset.shape)

(999, 256, 256, 1)


In [ ]:
image_dataset=image_dataset/255
mask_dataset=mask_dataset/255

print("Image data shape is: ", image_dataset.shape)
print("Mask data shape is: ", mask_dataset.shape)
print("Max pixel value in image is: ", image_dataset.max())
print("Labels in the mask are : ", np.unique(mask_dataset))

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(image_dataset,mask_dataset, test_size=.2, random_state=42)

In [ ]:
#building U net
from keras.models import Model
from keras.layers.convolutional import  Conv2D, MaxPooling2D,UpSampling2D,Conv2DTranspose
from tensorflow.keras.optimizers import Adam,RMSprop
from keras.layers import Activation, MaxPooling2D, Concatenate,Input, BatchNormalization,Dropout,Lambda


def conv_block(input, num_filters):
    x=Conv2D(num_filters,3,padding="same")(input)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)


    x=Conv2D(num_filters,3,padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    return x



def encoder_block(input, num_filters):
    x=conv_block(input,num_filters)
    p=MaxPooling2D((2,2))(x)

    return x,p




def decoder_block(input,skip_feature, num_filters):
    x=Conv2DTranspose(num_filters,(2,2),strides=2, padding="same")(input)
    x=Concatenate()([x,skip_feature])
    x=conv_block(x,num_filters)
    return x




def build_unet(input_shape, n_classes):
    inputs=Input(input_shape)

    s1,p1=encoder_block(inputs,64)
    s2,p2=encoder_block(p1,128)
    s3,p3=encoder_block(p2,256)
    s4,p4=encoder_block(p3,512)


    b1=conv_block(p4,1024) # bridge


    d1=decoder_block(b1,s4,512)
    d2=decoder_block(d1,s3,256)
    d3=decoder_block(d2, s2,128)
    d4=decoder_block(d3,s1,64)

    if n_classes==1:
        activation="sigmoid"

    else:
        activation="softmax"



    outputs=Conv2D(n_classes,1,padding="same",activation=activation)(d4)


    model=Model(inputs,outputs,name="U-Net")

    return model


In [ ]:
Img_height=image_dataset.shape[1]
Img_width=image_dataset.shape[2]
Img_channel=image_dataset.shape[3]
input_shape=(Img_height,Img_width,Img_channel)

model=build_unet(input_shape,n_classes=1)
model.summary()


# All loss func

In [ ]:
from keras.losses import binary_crossentropy
import keras.backend as K
import tensorflow as tf

epsilon = 1e-5
smooth = 1

#Dice Loss
def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

#Binary Cross Entropy + Dice Loss
def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

#Tversky Loss
def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

#Focal Tversky Loss
def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)


#Combined
def hybrid_loss(y_true, y_pred):
    a = focal_tversky(y_true, y_pred)
    b = bce_dice_loss(y_true, y_pred)
    #result = tversky + _lambda_ * dice
    result = a + b
    return result
    return hybrid_loss

# Callback

In [ ]:
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)


2.11.0


In [ ]:
model_filepath="/content/drive/MyDrive/Colab Notebooks/Fetal/unet_improved.hdf5"
from tensorflow.keras.callbacks import ModelCheckpoint
cp_callback = ModelCheckpoint(filepath=model_filepath,
                                                 save_weights_only=True,
                                                 verbose=1)


# Model compile

In [ ]:
model.compile(optimizer=Adam(learning_rate = 2e-3), loss=hybrid_loss, metrics=[dsc])
model.summary()

## Data Augmentation

In [ ]:
data_gen_args = dict(
    #featurewise_center=True,
                     #featurewise_std_normalization=True,
                     rotation_range=90,
                     horizontal_flip=True,
                     vertical_flip=True,
                     #width_shift_range=0.1,
                     #height_shift_range=0.1,
                     #zoom_range=0.2,
                     fill_mode="reflect"
                    )
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

data_gen_arg = dict(featurewise_center=True,
                     #featurewise_std_normalization=True,
                     rotation_range=90,
                     horizontal_flip=True,
                     vertical_flip=True,
                     #width_shift_range=0.1,
                     #height_shift_range=0.1,
                     #zoom_range=0.2
                    )
test_image_datagen = ImageDataGenerator(**data_gen_arg)
test_mask_datagen = ImageDataGenerator(**data_gen_arg)

In [ ]:
batchsize=25
# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(X_train, augment=True, seed=seed)
mask_datagen.fit(y_train, augment=True, seed=seed)
image_generator = image_datagen.flow(
    X_train,
    batch_size=batchsize,
    #class_mode=None,
    seed=seed)
mask_generator = mask_datagen.flow(
    y_train,
    #class_mode=None,
    batch_size=batchsize,
    seed=seed)

image_val = test_image_datagen.flow(
    X_test,
    #class_mode=None,
    seed=seed)
mask_val = test_mask_datagen.flow(
    y_test,
    #class_mode=None,
    seed=seed)


In [ ]:
#from keras.callbacks import EarlyStopping
#from keras.callbacks import ModelCheckpoint
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
#mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# Model Fit

In [ ]:
# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)
val_generator= zip(X_test,y_test)

history=model.fit(
    train_generator,
    validation_data=(X_test,y_test),
    batch_size=4,
    steps_per_epoch=30,
    epochs=100,
    #callbacks=[es, mc]
    #callbacks=[cp_callback]
)

In [ ]:
#Save the model for future use
model.save('/content/drive/MyDrive/Colab Notebooks/saved_models/fetal.hdf5')

#plot the training and validation accuracy and dsc at each epoch
dsc = history.history['dsc']
val_dsc = history.history['val_dsc']
epochs = range(1, len(dsc) + 1)
plt.plot(epochs, dsc, 'y', label='Training Dice Similarity Coefficient')
plt.plot(epochs, val_dsc, 'r', label='Validation Dice Similarity Coefficient')
plt.title('Training and validation Dice Similarity Coefficient')
plt.xlabel('Epochs')
plt.ylabel('Dice Similarity Coefficient')
plt.legend()
plt.show()

# IoU

In [ ]:
#IOU
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.5
from tensorflow.keras.metrics import MeanIoU

n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)
IOU_keras.update_state(y_pred_thresholded, y_test)
print("Mean IoU =", IOU_keras.result().numpy())

# *Plotting predicted images*

In [ ]:
import random
threshold = 0.5
test_img_number = random.randint(0, len(X_test))
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]
test_img_input=np.expand_dims(test_img, 0)
print(test_img_input.shape)
prediction = (model.predict(test_img_input)[0,:,:,0]> 0.14).astype("uint8")
print(prediction.shape)
#plt.title('Testing Image')
plt.imshow(np.squeeze(test_img),cmap='gray')
plt.axis('off')

In [ ]:
plt.title('Prediction on test image')
#plt.imshow(prediction, cmap='gray')
plt.imshow(np.squeeze(prediction),cmap='gray')
plt.axis('off')

In [ ]:
plt.title('Annotated Mask')
plt.imshow(np.squeeze(ground_truth),cmap='gray')
plt.axis('off')

# Superimposition of images

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
# Reading the images that are present
source1 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Fetal/head 3.png', cv2.IMREAD_COLOR)
source2 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Fetal/pred3.png', cv2.IMREAD_COLOR)
# blending the image by adding necessary weights
a=cv2.resize(source1,(256,256))
b=cv2.resize(source2,(256,256))
# blending the image by adding necessary weights
dest = cv2.addWeighted(a, 0.5, b, 0.5, 0.0)
#dest = cv2.addWeighted(source1, 0.5, source2, 0.5, 0.0)
# saving the output of the image
#cv2.imwrite('img.png', dest)
cv2_imshow(dest)
# Wait for a key
cv2.waitKey(0)
# Distroy all the window open
cv2.destroyAllWindows()
plt.axis('off')

In [ ]:
cv2_imshow(a)

In [ ]:
cv2_imshow(b)

# Original U Net

Model compile

In [ ]:
IMG_HEIGHT = image_dataset.shape[1]
IMG_WIDTH  = image_dataset.shape[2]
IMG_CHANNELS = image_dataset.shape[3]

input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

model = build_unet(input_shape, n_classes=1)
model.compile(optimizer=Adam(learning_rate = 1e-3), loss=binary_crossentropy, metrics=[dsc])
model.summary()


In [ ]:
history = model.fit(X_train, y_train,
                    batch_size = 4,
                    verbose=1,
                    epochs=100,
                    validation_data=(X_test, y_test),
                    shuffle=False)

In [ ]:
#Save the model for future use
model.save('/content/drive/MyDrive/Colab Notebooks/saved_models/fetal.hdf5')

#plot the training and validation dsc at each epoch
dsc = history.history['dsc']
val_dsc = history.history['val_dsc']
epochs = range(1, len(dsc) + 1)
plt.plot(epochs, dsc, 'y', label='Training Dice Similarity Coefficient')
plt.plot(epochs, val_dsc, 'r', label='Validation Dice Similarity Coefficient')
plt.title('Training and validation Dice Similarity Coefficient')
plt.xlabel('Epochs')
plt.ylabel('Dice Similarity Coefficient')
plt.legend()
plt.show()

Prediction from original U net

In [ ]:
import random
threshold = 0.5
test_img_number = random.randint(0, len(X_test))
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]
test_img_input=np.expand_dims(test_img, 0)
print(test_img_input.shape)
prediction = (model.predict(test_img_input)[0,:,:,0]> 0.14).astype("uint8")
print(prediction.shape)

plt.figure(figsize=(16, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(np.squeeze(test_img),cmap='gray')
plt.axis('off')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(np.squeeze(ground_truth),cmap='gray')
plt.axis('off')
plt.subplot(233)
plt.title('Prediction on test image')
#plt.imshow(prediction, cmap='gray')
plt.imshow(np.squeeze(prediction),cmap='gray')
plt.axis('off')
plt.show()